In [1]:
import pandas as pd
import numpy as np

# Find representative producer countries

In [2]:
df = pd.read_excel('imap_export.xls', header=1)
df

,Factory Name,Factory Type,Product Type Type,"Nike, Inc. Brand(s)",Events,Supplier Group,Address,City,State,Postal Code,Country / Region,Region,Total Workers,Line Workers,% Female Workers,% Migrant Workers
0,"A & K Designs, Inc.",FINISHED GOODS,Apparel,Nike,NaN,A & K DESIGNS,8564 NE Alderwood Road,Portland,Oregon,97220,USA,AMERICAS,111,95,73,0
1,"ACode Sporting Goods Co., Ltd.",FINISHED GOODS,Equipment,Nike,NaN,EXCELLENCE SPORTING GOODS,No 32 VSIP II A Street 31,Bac Tan Uyen,Bình Duong,822710,Vietnam,SE ASIA,347,318,79,0
2,Ad Dulyal,FINISHED GOODS,Apparel,Nike,NaN,MAS HOLDINGS,Part of land no 1075 Basin 5 Ad Dulayl,Zarqa,Az Zarqa,11183,Jordan,EMEA,1303,1166,87,75
3,ADORA FOOTWEAR LIMITED,FINISHED GOODS,Footwear,Converse,NaN,HUALI,TAM DIEP INDUSTRY ZONE,Ninh Binh Province,Ninh Bình,430000,Vietnam,SE ASIA,8120,6800,86,0
4,"AHP APPAREL PVT LTD., UNIT 60",FINISHED GOODS,Apparel,Nike,Collegiate,SHAHI,207 ABDE F KIADB INDUSTRIAL AREA,HASSAN,Karnataka,573201,India,S ASIA,2704,906,75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,LELABPLUS,FINISHED GOODS,Apparel,Nike,NaN,LELAB,1 bis rue Jean le Galleu,Ivry sur Seine,Île-de-France,94200,France,EMEA,21,14,62,100
639,"Haivina Co., Ltd.",FINISHED GOODS,Equipment,Nike,NaN,HAIVINA,Lang Xuyen village,Hai Duong province,Hi Duong,170000,Vietnam,SE ASIA,3259,2870,96,0
640,Tower Garments (London) Ltd,FINISHED GOODS,Apparel,Nike,NaN,TOWER GARMENTS,Unit J 17 Queensway Enfield,ENFIELD,London-- City of,EN3 4SA,United Kingdom,EMEA,90,90,60,0
641,"QINGDAO HONGTAISHENGDA TRADE CO., LTD.",FINISHED GOODS - COMPONENTS,Footwear,Nike,NaN,"QINGDAO HONGTAISHENGDA TRADE CO., LTD.",Beijingxi Road 41,Qingdao,Shandong,266300,China,N ASIA,100,80,90,0


In [3]:
workers_per_country = df.groupby(['Region', 'Country / Region']).sum()['Total Workers'].reset_index().sort_values(['Region', 'Total Workers'], ascending=False)
workers_per_country

,Region,Country / Region,Total Workers
37,SE ASIA,Vietnam,509169
35,SE ASIA,Cambodia,56673
36,SE ASIA,Thailand,26432
31,S ASIA,Indonesia,279988
33,S ASIA,Pakistan,44785
30,S ASIA,India,43407
34,S ASIA,Sri Lanka,38311
32,S ASIA,Malaysia,7508
25,N ASIA,China,143361
29,N ASIA,Taiwan,12714


In [4]:
# Representative producer countries for each region
top_countries = workers_per_country.groupby('Region').head(1)
top_countries

,Region,Country / Region,Total Workers
37,SE ASIA,Vietnam,509169
31,S ASIA,Indonesia,279988
25,N ASIA,China,143361
12,EMEA,Egypt,12462
1,AMERICAS,Brazil,21376


# Find representative consumer countries for each region
- For top countries by consumption, we choose countries by GDP (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal), sorted by forecast).
    - AMERICAS - US, N ASIA - China, EMEA - Germany, S ASIA - India, SE Asia - Japan.
- Ports for reference: largest port in the country (UN/LOCODE)
    - US: Port of Los Angeles (USLAX)
    - China: Port of Shanghai (CNSGH / CNSHA)
    - Germany: Hamburg Port (DEHAM)
    - India: Mundra (INMUN)
    - Japan: Tokyo (JPTYO)
    - Vietnam: Hai Phong (VNSIT)
    - Indonesia: Tanjung Priok (IDTPR / IDTPP)
    - Egypt: Port Said (EGPSD)
    - Brazil: Santos (BRSSZ)

# Estimating shipping cost from producer region to consumer region
- We use the Drewry World Container Index (https://en.macromicro.me/collections/4356/freight/44756/drewry-world-container-index) to get the overall trend of shipping cost.
- Drewry tracks the freight costs of 40-foot container via eight major routes, including spot rates and short-term contract rates.
- We take the cost in the first week of July, which is a good proxy for the average cost. 
- Because CMA-CGM and Freightos shipping costs are not always available between countries, we take a distance metric instead (https://www.geodatos.net/en/distances/countries).
- We see that the Drewry World Container Index is very similar to the Drewry World Container Index - Shanghai to LA, and so we use that as a basis.
- I checked the cost from Shanghai to LA on Freightos for a forty-foot and the cost is (exepctedly) very similar, so we can use the index directly.

In [5]:
years = np.arange(2014, 2024)
# Get Drewry World Container Index cost
global_shipping_costs = pd.DataFrame(
    {'year': years, # 10 years, 2014-2023
     'global_shipping_cost': [2053.72, 1583.27, 1427.14, 1549.72, 1468.18, 1372.11, 2031.57, 8399.09, 7050.94, 1474.32]})
global_shipping_costs

,year,global_shipping_cost
0,2014,2053.72
1,2015,1583.27
2,2016,1427.14
3,2017,1549.72
4,2018,1468.18
5,2019,1372.11
6,2020,2031.57
7,2021,8399.09
8,2022,7050.94
9,2023,1474.32


In [6]:
# Get distances (km)
data = {
    # ['SE ASIA (Vietnam)', 'S ASIA (Indonesia)', 'N ASIA (China)', 'EMEA (Egypt)', 'AMERICAS (Brazil)']
    'SE ASIA (Japan)': [3864, 4808, 3054, 9704, 17371],
    'S ASIA (India)': [3194, 4483, 2984, 4931, 14787],
    'N ASIA (China)': [2450, 4181, 0, 6896, 16638],
    'EMEA (Germany)': [9339, 11014, 7242, 3203, 9417],
    'AMERICAS (US)': [13814, 14972, 11671, 10997, 7301]
}

# Rows (representative producer country)
rows = ['SE ASIA (Vietnam)', 'S ASIA (Indonesia)', 'N ASIA (China)', 'EMEA (Egypt)', 'AMERICAS (Brazil)']

# Create df
distances = pd.DataFrame(data, index=rows)
distances

,SE ASIA (Japan),S ASIA (India),N ASIA (China),EMEA (Germany),AMERICAS (US)
SE ASIA (Vietnam),3864,3194,2450,9339,13814
S ASIA (Indonesia),4808,4483,4181,11014,14972
N ASIA (China),3054,2984,0,7242,11671
EMEA (Egypt),9704,4931,6896,3203,10997
AMERICAS (Brazil),17371,14787,16638,9417,7301


In [7]:
# Normalize distances such that the distance from China to US is 1
shipping_costs = distances / 11671
shipping_costs = shipping_costs.reset_index().melt(id_vars=['index'], var_name='Column', value_name='Value')
shipping_costs = shipping_costs.rename(columns={'index':'producer_region', 'Column':'consumer_region', 'Value':'distance'})
shipping_costs

,producer_region,consumer_region,distance
0,SE ASIA (Vietnam),SE ASIA (Japan),0.331077
1,S ASIA (Indonesia),SE ASIA (Japan),0.411961
2,N ASIA (China),SE ASIA (Japan),0.261674
3,EMEA (Egypt),SE ASIA (Japan),0.831463
4,AMERICAS (Brazil),SE ASIA (Japan),1.488390
5,SE ASIA (Vietnam),S ASIA (India),0.273670
6,S ASIA (Indonesia),S ASIA (India),0.384114
7,N ASIA (China),S ASIA (India),0.255676
8,EMEA (Egypt),S ASIA (India),0.422500
9,AMERICAS (Brazil),S ASIA (India),1.266987


In [8]:
# Estimate number of items that can fit in a 40-foot (40') container
# A Nike shoe box measures 14 x 7.5 x 5 inches (14" x 7.5" x 7") (https://www.kusashoes.com/how-big-is-a-nike-shoe-box/)
# Using this calculator (https://www.gigacalculator.com/calculators/container-loading-calculator.php), 
# we get an estimate of 5,148 shoe boxes per container.
quantity_per_container = 5148

In [9]:
# Multiply by global_shipping_cost to get the total shipping cost (for each year)
# Then divide by 5,148 to get the shipping cost per item (for each year)
for i, year in enumerate(years):
    shipping_costs[f'shipping_cost_{str(year)}'] = shipping_costs['distance'] * global_shipping_costs.iloc[i,1] / quantity_per_container
shipping_costs

,producer_region,consumer_region,distance,shipping_cost_2014,shipping_cost_2015,shipping_cost_2016,shipping_cost_2017,shipping_cost_2018,shipping_cost_2019,shipping_cost_2020,shipping_cost_2021,shipping_cost_2022,shipping_cost_2023
0,SE ASIA (Vietnam),SE ASIA (Japan),0.331077,0.132078,0.101823,0.091782,0.099665,0.094421,0.088243,0.130654,0.540160,0.453458,0.094816
1,S ASIA (Indonesia),SE ASIA (Japan),0.411961,0.164346,0.126699,0.114205,0.124014,0.117489,0.109801,0.162573,0.672125,0.564241,0.117980
2,N ASIA (China),SE ASIA (Japan),0.261674,0.104391,0.080478,0.072542,0.078773,0.074628,0.069745,0.103265,0.426928,0.358401,0.074940
3,EMEA (Egypt),SE ASIA (Japan),0.831463,0.331700,0.255717,0.230500,0.250298,0.237128,0.221612,0.328122,1.356552,1.138810,0.238120
4,AMERICAS (Brazil),SE ASIA (Japan),1.488390,0.593772,0.457755,0.412615,0.448055,0.424480,0.396705,0.587368,2.428345,2.038568,0.426255
5,SE ASIA (Vietnam),S ASIA (India),0.273670,0.109177,0.084167,0.075867,0.082384,0.078049,0.072942,0.107999,0.446499,0.374831,0.078375
6,S ASIA (Indonesia),S ASIA (India),0.384114,0.153237,0.118135,0.106485,0.115631,0.109547,0.102379,0.151584,0.626692,0.526101,0.110005
7,N ASIA (China),S ASIA (India),0.255676,0.101998,0.078633,0.070879,0.076967,0.072917,0.068146,0.100898,0.417143,0.350186,0.073222
8,EMEA (Egypt),S ASIA (India),0.422500,0.168550,0.129940,0.117126,0.127187,0.120495,0.112610,0.166732,0.689320,0.578676,0.120999
9,AMERICAS (Brazil),S ASIA (India),1.266987,0.505446,0.389662,0.351237,0.381405,0.361337,0.337693,0.499995,2.067120,1.735324,0.362848


### Comments
The shipping cost is typically $0.10-$0.50 per pair of shoes, but spiked to $0.50-$2 during Covid-19 (2021-2022). Looks reasonable.

In [10]:
# Save as csv file
shipping_costs.to_csv('shipping_costs.csv', index=False)